# Part 1: Computer Vision Functions
***
# Table of Contents
## Global
1.   [Setup](#Setup)

## Stage 1
1.   [Object Extraction](#Object-Extraction)
2.   [Filtering](#Filtering)
3.   [Object Blending](#Object-Blending)
4.   [Result Comparison](#Result-Comparison)

## Stage 2
1.   [Remove Green](#Remove-Green)
2.   [Change Background](#Change-Background)

# Setup

3 libraries are used for this part of the assignment:
* [opencv](https://opencv.org/)
* [numpy](https://numpy.org/)
* [tqdm](https://tqdm.github.io/)

In [1]:
import cv2
print('cv2 version:', cv2.__version__)
import numpy as np
print('numpy version:', np.__version__)
import tqdm
print('tqdm version:', tqdm.__version__)
from tqdm.notebook import tqdm

cv2 version: 4.0.1
numpy version: 1.19.2
tqdm version: 4.59.0


# Object Extraction

Given a scene containing objects and a mask of a desired object, the ```ExtractObject``` function will extract the object from the image.

In [2]:
def ExtractObject(S2, ObjectMask):
    return cv2.bitwise_and(S2, S2, mask=ObjectMask)
    

S2 = cv2.imread("Images/2_colour.jpeg", 1)

ObjectMask = cv2.imread("Masks/statues_no_3_colour_mask_2_mask.png", 0)

ExtractedObject = ExtractObject(S2, ObjectMask)
cv2.imwrite("Output/Extracted Object.png", ExtractedObject, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

# Filtering

3 filters are provided with the ```ApplyFilter``` function:

* 2D Convolution 5x5 averaging
* Gaussian Blurring
* Sharpening Kernel

In [3]:
def ApplyFilter(ExtractedObject, FilterIndex):
    if FilterIndex == 0:
        return ExtractedObject
    
    if FilterIndex == 1:
        # 2D Convolution 5x5 averaging
        kernel = np.ones((5,5),np.float32)/25
        return cv2.filter2D(ExtractedObject, -1, kernel)
    
    if FilterIndex == 2:
        # Gaussian Blurring
        return cv2.GaussianBlur(ExtractedObject, (5,5), 0)
    
    if FilterIndex == 3:
        # Sharpening Kernel
        kernel = np.array([[-1,-1,-1],
                       [-1, 9,-1],
                       [-1,-1,-1]])
        return cv2.filter2D(ExtractedObject, -1, kernel)

cv2.imwrite("Output/Filtered Object 0.png", ApplyFilter(ExtractedObject.copy(), 0), [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imwrite("Output/Filtered Object 1.png", ApplyFilter(ExtractedObject.copy(), 1), [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imwrite("Output/Filtered Object 2.png", ApplyFilter(ExtractedObject.copy(), 2), [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imwrite("Output/Filtered Object 3.png", ApplyFilter(ExtractedObject.copy(), 3), [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

# Object Blending

In [4]:
def ObjectBlending(S1, FilteredExObject):
    
    c_text = S1.copy()
    for i, y in enumerate(FilteredExObject):
        for j, x in enumerate(y):
            if np.sum(FilteredExObject[i][j]) != 0:
                c_text[i][j] = x 
    return c_text

S1 = cv2.imread("Images/1_colour.jpeg")
BlendingResult = ObjectBlending(S1, ExtractedObject)
cv2.imwrite("Output/Blended Result.png", BlendingResult, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

# Result Comparison

The Sum of Squared Error (SSE) and Mean Squared Error (MSE) are used as metrics. In ```CompareResult``` the metric parameter distinguishes between the 2.

In [5]:
def SSD(image1, image2):
    return np.sum((image1.astype("float") - image2.astype("float")) ** 2)

def MSE(image1, image2):
    return np.mean((image1-image2)**2)

def CompareResult(BlendingResult, S2, metric):
    if metric == 0:
        return MSE(BlendingResult, S2)
    else:
        return SSD(BlendingResult,S2)

error = CompareResult(BlendingResult, S2, 0)
print(error)

58.45912688078704


# Remove Green

The ```removeGreen``` function removes the green background from an image, making sure to keep green colour inside the objects in an image.

In [6]:
def removeGreen(img):
    # convert to hsv
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # mask of green (36,25,25) ~ (86, 255,255)
    mask = cv2.inRange(hsv, (36, 25, 25), (86, 255,255))

    # slice the green
    imask = mask > 0
    green = np.zeros_like(img, np.uint8)
    green[imask] = img[imask]
    cv2.imwrite("Output/only Green.png", green, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # flip to gray
    gray = cv2.cvtColor(green.copy(),cv2.COLOR_BGR2GRAY)
    cv2.imwrite("Output/gray.png", gray, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # get treshold
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    cv2.imwrite("Output/tresh.png", thresh, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # flip
    flip = cv2.bitwise_not(thresh)
    cv2.imwrite("Output/flip.png", flip, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    kernel = np.ones((20, 20), 'uint8')
    close = cv2.morphologyEx(flip, cv2.MORPH_CLOSE ,kernel)
    cv2.imwrite("Output/close.png", close, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    c_img = img.copy()
    for i, y in enumerate(img):
        for j, x in enumerate(y):
            if np.sum(close[i][j]) != 255:
                c_img[i][j] = (0, 0, 0)
    return c_img

img = cv2.imread("Images/Books/1_colour.jpeg")
no_green = removeGreen(img.copy())
cv2.imwrite("Output/No Green.png", no_green, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

# Change background

The ```changeBackground``` function uses the modified code from ```removeGreen```. The main difference here is that the final step of repainting the object onto the return image is that a new provided background is used instead of black.

In [7]:
def removeGreen(img):
    # convert to hsv
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # mask of green (36,25,25) ~ (86, 255,255)
    mask = cv2.inRange(hsv, (36, 25, 25), (86, 255,255))

    # slice the green
    imask = mask > 0
    green = np.zeros_like(img, np.uint8)
    green[imask] = img[imask]
    cv2.imwrite("Output/only Green.png", green, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # flip to gray
    gray = cv2.cvtColor(green.copy(),cv2.COLOR_BGR2GRAY)
    cv2.imwrite("Output/gray.png", gray, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # get treshold
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    cv2.imwrite("Output/tresh.png", thresh, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # flip
    flip = cv2.bitwise_not(thresh)
    cv2.imwrite("Output/flip.png", flip, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    kernel = np.ones((20, 20), 'uint8')
    close = cv2.morphologyEx(flip, cv2.MORPH_CLOSE ,kernel)
    cv2.imwrite("Output/close.png", close, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    c_img = img.copy()
    for i, y in enumerate(img):
        for j, x in enumerate(y):
            if np.sum(close[i][j]) != 255:
                c_img[i][j] = (0, 0, 0)
    return c_img

img = cv2.imread("Images/Books/1_colour.jpeg")
no_green = removeGreen(img.copy())
cv2.imwrite("Output/No Green.png", no_green, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

# Change background

In [8]:
def changeBackground(img, background):
    assert img.shape[0] <=  background.shape[0] and img.shape[1] <=  background.shape[1]

    # convert to hsv
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # mask of green (36,25,25) ~ (86, 255,255)
    mask = cv2.inRange(hsv, (36, 25, 25), (86, 255,255))

    # slice the green
    imask = mask > 0
    green = np.zeros_like(img, np.uint8)
    green[imask] = img[imask]

    # flip to gray
    gray = cv2.cvtColor(green.copy(),cv2.COLOR_BGR2GRAY)

    # get treshold
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)

    # flip
    flip = cv2.bitwise_not(thresh)

    kernel = np.ones((20, 20), 'uint8')
    close = cv2.morphologyEx(flip, cv2.MORPH_CLOSE ,kernel)

    c_img = background.copy()
    for i, y in tqdm(enumerate(background)):
        if i < img.shape[0]:
            for j, x in enumerate(y):
                if j < img.shape[1]:
                    if np.sum(close[i][j]) == 255:
                        c_img[i][j] = img[i][j]
    return c_img

book = cv2.imread("Images/Books/1_colour.jpeg")
jesus = cv2.imread("Images/2_colour.jpeg")
buddha = cv2.imread("Images/5_colour.jpeg")
backgroundLarge = cv2.imread("Images/Ben 1920.png")
backgroundExact = cv2.imread("Images/Ben 720.png")
backgroundDog = cv2.imread("Images/Dog 720.png")
backgroundJake = cv2.imread("Images/Jake 720.png")
new_background = changeBackground(img.copy(), backgroundExact)
cv2.imwrite("Output/On Exact Background.png", new_background, [cv2.IMWRITE_PNG_COMPRESSION, 0])
new_background = changeBackground(img.copy(), backgroundLarge)
cv2.imwrite("Output/On Large Background.png", new_background, [cv2.IMWRITE_PNG_COMPRESSION, 0])
new_background = changeBackground(jesus.copy(), backgroundDog)
cv2.imwrite("Output/On Dog.png", new_background, [cv2.IMWRITE_PNG_COMPRESSION, 0])
new_background = changeBackground(buddha.copy(), backgroundJake)
cv2.imwrite("Output/On Jake.png", new_background, [cv2.IMWRITE_PNG_COMPRESSION, 0])

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

True